In [ ]:
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import pandas as pd
import importlib
import sys

# Determine root directory - go up from notebooks folder to portfolio_analyser
if Path.cwd().name == 'notebooks':
    ROOT = Path.cwd().parent
else:
    ROOT = Path.cwd()

src_path = ROOT / 'src'

# Add src to path if not already there
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import analysis and reporting functions
import technical_analysis as ta
from portfolio_reports import create_trading_playbook_pdf, create_portfolio_tracker_excel, cleanup_old_reports

# Setup
RESULTS_DIR = ROOT / 'portfolio_results'
RESULTS_DIR.mkdir(exist_ok=True)
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')

In [2]:
# ===================================================================================================
# CELL 2: LOAD PORTFOLIO DATA
# ===================================================================================================

# Load holdings and targets from data folder
holdings_file = ROOT / 'data' / 'holdings.csv'
targets_file = ROOT / 'data' / 'targets.csv'
stocks_file = ROOT / 'data' / 'stocks.txt'

# Read holdings
if not holdings_file.exists():
    print(f"Creating new holdings file: {holdings_file}")
    holdings_df = pd.DataFrame(columns=['ticker', 'quantity', 'avg_cost'])
    holdings_df.to_csv(holdings_file, index=False)
else:
    holdings_df = pd.read_csv(holdings_file)

# Read targets
if not targets_file.exists():
    print(f"Error: {targets_file} not found")
    raise FileNotFoundError(f"targets.csv required")

targets_df = pd.read_csv(targets_file)

# Convert target_pct from 0-100 to 0-1 if needed
if targets_df['target_pct'].max() > 1.0:
    targets_df['target_pct'] = targets_df['target_pct'] / 100.0

# Get list of individual tickers (no baskets)
baskets = {
    'BTC': ['BTC-USD']
}

individual_tickers = []
for ticker in targets_df['ticker']:
    if ticker in baskets:
        individual_tickers.extend(baskets[ticker])
    else:
        individual_tickers.append(ticker)

print(f"✓ Loaded {len(holdings_df)} holdings")
print(f"✓ Loaded {len(targets_df)} target allocations")
print(f"✓ Expanded to {len(individual_tickers)} individual tickers")
print(f"\nHoldings:\n{holdings_df.to_string()}")

✓ Loaded 13 holdings
✓ Loaded 13 target allocations
✓ Expanded to 13 individual tickers

Holdings:
     ticker  quantity  avg_cost last_updated  min_quantity
0      TSLA      72.0     310.6   2026-01-04           0.0
1      NVDA       0.0       0.0          NaN           0.0
2      MSFT       0.0       0.0          NaN           0.0
3      META       0.0       0.0          NaN           0.0
4      PLTR       0.0       0.0          NaN           0.0
5      MSTR       0.0       0.0          NaN           0.0
6      ASML       0.0       0.0          NaN           0.0
7       AMD       0.0       0.0          NaN           0.0
8      AVGO       0.0       0.0          NaN           0.0
9      ALAB       0.0       0.0          NaN           0.0
10     MRVL       0.0       0.0          NaN           0.0
11  BTC-USD       0.5   50090.1   2026-01-04           0.5
12  SOL-USD       0.0       0.0          NaN           0.0


In [3]:
# Set actual cash available
CASH_AVAILABLE = 110439.86

# Calculate initial portfolio total from holdings at avg_cost
PORTFOLIO_TOTAL = 0
initial_holdings_value = 0

for _, row in holdings_df.iterrows():
    quantity = row['quantity']
    if quantity > 0:
        initial_holdings_value += quantity * row['avg_cost']

PORTFOLIO_TOTAL = initial_holdings_value

print(f"Cash Available: ${CASH_AVAILABLE:,.2f}")
print(f"Initial Holdings Value (avg cost): ${initial_holdings_value:,.2f}")
print(f"Initial Portfolio Total: ${PORTFOLIO_TOTAL:,.2f}")

# Calculate total target value (aspirational)
total_target_value = targets_df['target_value'].sum()
print(f"Total Target Value (aspirational): ${total_target_value:,.2f}")

Cash Available: $110,439.86
Initial Holdings Value (avg cost): $47,408.25
Initial Portfolio Total: $47,408.25
Total Target Value (aspirational): $200,030.00


In [4]:
# ===================================================================================================
# CELL 3: ANALYZE PORTFOLIO STOCKS
# ===================================================================================================

print("Starting portfolio analysis...")
print(f"Stocks to analyze: {len(individual_tickers)}")
print(f"Concurrency: 2 threads")
print(f"Cache: 24hr TTL, 1-2s delay between API calls\n")

results = []
errors = []

def analyze_stock(ticker):
    """Analyze single stock with error handling"""
    try:
        result = ta.analyze_ticker(ticker)
        if result:
            return ticker, result, None
    except Exception as e:
        return ticker, None, str(e)
    return ticker, None, "Unknown error"

with ThreadPoolExecutor(max_workers=2) as executor:
    futures = {executor.submit(analyze_stock, ticker): ticker for ticker in individual_tickers}

    for future in as_completed(futures):
        ticker, result, error = future.result()

        if result:
            results.append(result)
            print(f"✓ {ticker}: {result['signal']}")
        else:
            errors.append((ticker, error))
            print(f"✗ {ticker}: {error}")

print(f"\n{'='*80}")
print(f"Analysis complete: {len(results)} success, {len(errors)} errors")
print(f"{'='*80}")

Starting portfolio analysis...
Stocks to analyze: 13
Concurrency: 2 threads
Cache: 24hr TTL, 1-2s delay between API calls

✗ TSLA: name 'ta' is not defined
✗ NVDA: name 'ta' is not defined
✗ MSTR: name 'ta' is not defined
✗ PLTR: name 'ta' is not defined
✗ META: name 'ta' is not defined
✗ AMD: name 'ta' is not defined
✗ AVGO: name 'ta' is not defined
✗ ALAB: name 'ta' is not defined
✗ MRVL: name 'ta' is not defined
✗ SOL-USD: name 'ta' is not defined
✗ BTC-USD: name 'ta' is not defined
✗ MSFT: name 'ta' is not defined
✗ ASML: name 'ta' is not defined

Analysis complete: 0 success, 13 errors


In [5]:
# Recalculate PORTFOLIO_TOTAL using current prices from results
PORTFOLIO_TOTAL_CURRENT = 0

print("\nRecalculating portfolio with current prices:")
for _, row in holdings_df.iterrows():
    ticker = row['ticker']
    quantity = row['quantity']
    if quantity > 0:
        # Find the result for this ticker
        result = next((r for r in results if r['ticker'] == ticker), None)
        if result:
            current_price = result['current_price']
            current_value = quantity * current_price
            PORTFOLIO_TOTAL_CURRENT += current_value
            print(f"{ticker}: {quantity:.4f} quantity @ ${current_price:.2f} = ${current_value:,.2f}")

PORTFOLIO_TOTAL = PORTFOLIO_TOTAL_CURRENT
print(f"\n✓ Updated PORTFOLIO_TOTAL = ${PORTFOLIO_TOTAL:,.2f}")


Recalculating portfolio with current prices:

✓ Updated PORTFOLIO_TOTAL = $0.00


In [6]:
# ===================================================================================================
# CELL 7: BUILD PORTFOLIO DATA WITH POSITION GAPS & TRANCHES
# ===================================================================================================

portfolio_positions = []
buy_count = 0
sell_count = 0
hold_count = 0

for result in results:
    if result is None:
        continue

    ticker = result['ticker']
    error = result.get('error')

    if error:
        errors.append(f"{ticker}: {error}")
        continue

    # Get holdings
    holding = holdings_df[holdings_df['ticker'] == ticker]
    quantity = holding['quantity'].iloc[0] if not holding.empty else 0
    min_quantity = holding['min_quantity'].iloc[0] if not holding.empty else 0

    # Calculate tradeable quantity (what we can actually sell)
    tradeable_quantity = max(0, quantity - min_quantity)

    # Get current values
    current_price = result['current_price']
    current_value = quantity * current_price
    tradeable_value = tradeable_quantity * current_price
    min_value = min_quantity * current_price

    # Get targets
    target = targets_df[targets_df['ticker'] == ticker]
    target_pct = target['target_pct'].iloc[0] if not target.empty else 0.0

    # Calculate position gap using CURRENT portfolio total
    position_gap = ta.calculate_position_gap(
        current_value=tradeable_value,  # Use tradeable value for gap calculations
        target_pct=target_pct,
        portfolio_total=PORTFOLIO_TOTAL_CURRENT + CASH_AVAILABLE
    )
    gap_value = position_gap['gap_value']
    current_pct = position_gap['current_pct']

    # Adjust sell levels for MAs blocking resistance
    adjusted_r1, adjusted_r2, adjusted_r3, ma_note = ta.adjust_sell_levels_for_mas(
        d50=result.get('d50'),
        d100=result.get('d100'),
        d200=result.get('d200'),
        r1=result.get('r1'),
        r2=result.get('r2'),
        r3=result.get('r3'),
        current_price=current_price
    )

    # Determine portfolio action
    action = ta.determine_portfolio_action(
        signal=result['signal'],
        position_gap=position_gap,
        buy_quality=result.get('buy_quality', 'N/A')
    )

    # Count actions
    if action == "BUY":
        buy_count += 1
    elif action == "SELL":
        sell_count += 1
    else:
        hold_count += 1

    # Calculate buy tranches
    buy_tranches = ta.calculate_buy_tranches(
        gap_value=gap_value,
        s1=result.get('s1'),
        s2=result.get('s2'),
        s3=result.get('s3'),
        current_price=result['current_price'],
        buy_quality=result.get('buy_quality', 'N/A')
    )

    # Calculate sell tranches (use tradeable_value since we can only sell tradeable quantity)
    sell_tranches = ta.calculate_sell_tranches(
        current_value=tradeable_value,
        signal=result['signal'],
        r1=result.get('r1'),
        r2=result.get('r2'),
        r3=result.get('r3'),
        adjusted_r1=adjusted_r1,
        adjusted_r2=adjusted_r2,
        adjusted_r3=adjusted_r3
    )

    # Build position dict
    position = {
        'ticker': ticker,
        'signal': result['signal'],
        'current_price': result['current_price'],
        'quantity': quantity,
        'min_quantity': min_quantity,
        'tradeable_quantity': tradeable_quantity,
        'current_value': current_value,
        'tradeable_value': tradeable_value,
        'min_value': min_value,
        'target_pct': target_pct,
        'current_pct': current_pct,
        'gap_value': gap_value,
        'action': action,
        'buy_quality': result.get('buy_quality', 'N/A'),
        'buy_quality_note': result.get('buy_quality_note', ''),
        'buy_tranches': buy_tranches,
        'sell_tranches': sell_tranches,
        # Technical levels for Technical Levels tab
        'd50': result.get('d50'),
        'd100': result.get('d100'),
        'd200': result.get('d200'),
        's1': result.get('s1'),
        's2': result.get('s2'),
        's3': result.get('s3'),
        'r1': result.get('r1'),
        'r2': result.get('r2'),
        'r3': result.get('r3')
    }

    portfolio_positions.append(position)

total_portfolio_value = PORTFOLIO_TOTAL + CASH_AVAILABLE
cash_pct_corrected = (CASH_AVAILABLE / total_portfolio_value) * 100

print(f"\nPortfolio Summary:")
print(f"  Total Value: ${total_portfolio_value:,.2f}")
print(f"  Current Holdings: ${PORTFOLIO_TOTAL:,.2f}")
print(f"  Cash Available: ${CASH_AVAILABLE:,.2f} ({cash_pct_corrected:.1f}%)")
print(f"\nAction Summary:")
print(f"  BUY signals: {buy_count}")
print(f"  SELL signals: {sell_count}")
print(f"  HOLD/WAIT: {hold_count}")


Portfolio Summary:
  Total Value: $110,439.86
  Current Holdings: $0.00
  Cash Available: $110,439.86 (100.0%)

Action Summary:
  BUY signals: 0
  SELL signals: 0
  HOLD/WAIT: 0


In [7]:
# ===================================================================================================
# CELL 6: PREPARE PORTFOLIO DATA
# ===================================================================================================

# Count actions
buy_count = sum(1 for p in portfolio_positions if p['action'] == 'BUY')
sell_count = sum(1 for p in portfolio_positions if p['action'] == 'SELL')
hold_count = sum(1 for p in portfolio_positions if p['action'] in ['HOLD', 'WAIT'])

# Calculate actual total portfolio value = current holdings + cash
total_portfolio_value = PORTFOLIO_TOTAL + CASH_AVAILABLE
cash_pct_corrected = (CASH_AVAILABLE / total_portfolio_value * 100) if total_portfolio_value > 0 else 0

# Build portfolio data structure
portfolio_data = {
    'portfolio_total': PORTFOLIO_TOTAL,
    'positions': portfolio_positions,
    'summary': {
        'buy_count': buy_count,
        'sell_count': sell_count,
        'hold_count': hold_count,
        'total_portfolio_value': total_portfolio_value,
        'cash_available': CASH_AVAILABLE
    }
}

print("Portfolio Summary:")
print(f"  Total Portfolio Value: ${total_portfolio_value:,.2f}")
print(f"  Current Holdings: ${PORTFOLIO_TOTAL:,.2f}")
print(f"  Cash Available: ${CASH_AVAILABLE:,.2f} (~{cash_pct_corrected:.0f}% dry powder)")
print(f"  Buy Actions: {buy_count}")
print(f"  Sell Actions: {sell_count}")
print(f"  Hold/Wait: {hold_count}")

Portfolio Summary:
  Total Portfolio Value: $110,439.86
  Current Holdings: $0.00
  Cash Available: $110,439.86 (~100% dry powder)
  Buy Actions: 0
  Sell Actions: 0
  Hold/Wait: 0


In [8]:
# ===================================================================================================
# CELL 6: GENERATE REPORTS
# ===================================================================================================

# Generate Trading Playbook PDF
print("\n=== Generating Trading Playbook PDF ===")
pdf_path = RESULTS_DIR / f'trading_playbook_{TIMESTAMP}.pdf'
create_trading_playbook_pdf(portfolio_data, pdf_path, TIMESTAMP)
print(f"✓ Created: {pdf_path.name}")

# Generate Portfolio Tracker Excel
print("\n=== Generating Portfolio Tracker Excel ===")
excel_path = RESULTS_DIR / f'portfolio_tracker_{TIMESTAMP}.xlsx'
create_portfolio_tracker_excel(portfolio_data, excel_path)
print(f"✓ Created: {excel_path.name}")

# Archive old reports
print("\n=== Cleaning Up Old Reports ===")
cleanup_old_reports(RESULTS_DIR, max_files=1)

print(f"\n{'='*80}")
print("✅ PORTFOLIO ANALYSIS COMPLETE")
print(f"{'='*80}")
print(f"\nGenerated Files:")
print(f"  1. {pdf_path.name}")
print(f"  2. {excel_path.name}")
print(f"\nLocation: {RESULTS_DIR.absolute()}")


=== Generating Trading Playbook PDF ===
✓ Created: trading_playbook_20260107_223631.pdf

=== Generating Portfolio Tracker Excel ===
✓ Created: portfolio_tracker_20260107_223631.xlsx

=== Cleaning Up Old Reports ===
  📦 Archived: trading_playbook_20260107_223540.pdf
  📦 Archived: portfolio_tracker_20260107_223540.xlsx
  ✅ Archived 2 file(s), kept 1 most recent

✅ PORTFOLIO ANALYSIS COMPLETE

Generated Files:
  1. trading_playbook_20260107_223631.pdf
  2. portfolio_tracker_20260107_223631.xlsx

Location: c:\workspace\portfolio_analyser\portfolio_results
